In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import torchvision.models as models

import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image

from tqdm import tqdm
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from torchcontrib.optim import SWA

import warnings
warnings.filterwarnings('ignore')

import gc

In [2]:
'''

학습 설정

추후에 argparser로 설정하면 될 듯

'''

train_data_dir = '../input/data/train/'
submission_data_dir = '../input/data/eval/'

train_image_dir = '../input/data/train/images'
submission_image_dir = '../input/data/eval/images'

model_dir = '../model/'

test_size = 0.2
fold_num = 5
seed = 22

lr = 0.01
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32
epochs = 10

image_size = (512, 384)
image_normal_mean = (0.5, 0.5, 0.5)
image_normal_std = (0.2, 0.2, 0.2)

In [3]:
df = pd.read_csv(train_data_dir + 'train.csv')
submission = pd.read_csv(submission_data_dir + 'info.csv')

In [4]:
'''

이상치 처리

'''

df.loc[df[df['id'] == '006359'].index, 'gender'] = 'male'
df.loc[df[df['id'] == '006359'].index, :]

,id,gender,race,age,path
2399,006359,male,Asian,18,006359_female_Asian_18


In [5]:
'''

학습 데이터셋 구축

'''
def get_age_group(x):
    if x < 30: return 0
    elif x < 60: return 1
    else: return 2

def get_gender_group(x):
    if x == 'male': return 0
    else: return 1

def get_train_df(df):
    train_df = []

    for idx, line in enumerate(tqdm(df.iloc)):
        for file in list(os.listdir(os.path.join(train_image_dir, line['path']))):
            if file[0] == '.':
                continue
            if file.split('.')[0] == 'normal':
                mask = 2
            elif file.split('.')[0] == 'incorrect_mask':
                mask = 1
            else:
                mask = 0
                
            mask_state = file.split('.')[0]
            gender_group = get_gender_group(line['gender'])
            age_group = get_age_group(line['age'])
            
            data = {
                'id' : line['id'],
                'gender' : line['gender'],
                'gender_group' : gender_group,
                'age' : line['age'],
                'age_group' : age_group,
                'mask_state': mask_state,
                'mask' : mask,
                'path': os.path.join(train_image_dir, line['path'], file),
                'label': mask * 6 + gender_group * 3 + age_group
            }
            
            train_df.append(data)
            
    train_df = pd.DataFrame(train_df)
    
    return train_df

train_df = get_train_df(df = df)

2700it [00:00, 2894.74it/s]


In [6]:
'''

Sample_submission 코드 참고

증강 데이터 생성은 본 함수에 추가할 것

데이터 셋 구축

'''

class CustomDataset(Dataset):
    def __init__(self, df, transform, train = True):
        self.train = train
        self.df = df
        if self.train:
            self.img_paths = self.df['path'].tolist()
            self.genders = self.df['gender_group'].tolist()
            self.ages = self.df['age_group'].tolist()
            self.masks = self.df['mask'].tolist()
            self.labels = self.df['label'].tolist()
        else:
            self.img_paths = [os.path.join(submission_image_dir, img_id) for img_id in self.df.ImageID]
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])
        if self.transform:
            image = self.transform(image)
        
        if self.train:
            label = torch.tensor(self.labels[index])
            age = torch.tensor(self.ages[index])
            gender = torch.tensor(self.genders[index])
            mask = torch.tensor(self.masks[index])
            return image, label, age, gender, mask
        
        else: return image

    def __len__(self):
        return len(self.img_paths)

In [7]:
def get_f1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

def get_acc_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

def model_train(model, optimizer, criterion, train_loder):
    model.train()
    
    total = 0
    train_loss = 0
    train_acc = 0
    train_fi_score = 0
    
    for images, labels, _, _, _ in train_loder:
        total += labels.size(0)
        
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        benign_outputs = model(images)
        loss = criterion(benign_outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
        
        _, predicted = benign_outputs.max(1)
        
        train_acc += predicted.eq(labels).sum().item()
        train_fi_score += get_f1_score(y_true = labels.cpu().tolist(), y_pred = predicted.cpu().tolist())
    
    train_loss /= len(train_loder)
    train_acc /= total
    train_fi_score /= len(train_loder)

    return train_loss, train_acc, train_fi_score


def model_eval(model, criterion, val_loder):
    model.eval()
    
    total = 0
    val_loss = 0
    val_acc = 0
    val_fi_score = 0
    
    with torch.no_grad():
        for images, labels, _, _, _ in val_loder:
            total += labels.size(0)

            images, labels = images.to(device), labels.to(device)

            benign_outputs = model(images)
            loss = criterion(benign_outputs, labels)

            val_loss += loss.item()

            _, predicted = benign_outputs.max(1)

            val_acc += predicted.eq(labels).sum().item()
            val_fi_score += get_f1_score(y_true = labels.cpu().tolist(), y_pred = predicted.cpu().tolist())
    
    val_loss /= len(val_loder)
    val_acc /= total
    val_fi_score /= len(val_loder)
   
    return val_loss, val_acc, val_fi_score

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [8]:
'''

데이터 전처리 함수

데이터 증강 등의 함수를 수정해서 사용하면 될 듯 함

'''

transform = transforms.Compose([
    Resize(image_size, Image.BILINEAR),
    ToTensor(),
    Normalize(mean=image_normal_mean, std=image_normal_std),
])

In [ ]:
'''

학습 시작

'''
total_start_time = time.time()

skf = StratifiedKFold(n_splits = fold_num, random_state = seed, shuffle = True)

criterion = nn.CrossEntropyLoss()

for now_fold_num, (trn_idx, val_idx) in tqdm(enumerate(skf.split(train_df, train_df['label']))):
    fold_start_time = time.time()
    
    # 데이터 정의
    trn_df = train_df.iloc[trn_idx, :]
    val_df = train_df.iloc[val_idx, :]
    
    trn_dataset = CustomDataset(df = trn_df, 
                                transform = transform, 
                                train = True)
    
    val_dataset = CustomDataset(df = val_df, 
                                transform = transform, 
                                train = True)
    
    train_loder = DataLoader(
    trn_dataset,
    batch_size = batch_size,
    shuffle=True,
    )
    
    val_loder = DataLoader(
    val_dataset,
    batch_size = batch_size,
    shuffle = False,
    )
    
    # 모델 정의
    model = models.efficientnet_b3(pretrained=True)
    in_features = model.classifier[1].in_features
    model.classifier[1] = torch.nn.Linear(in_features, 18)
    model = model.to(device)
    
    # optimizer 설정
    base_opt = torch.optim.Adam(model.parameters(), lr=lr)
    optimizer = SWA(base_opt, swa_start=3, swa_freq=2, swa_lr=0.005)
    
    # scheduler 설정
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor = 0.1, eps = 1e-09, patience = 5)
    
    # besf_metric 설정
    besf_fi = 0
    
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        
        # 학습
        train_loss, train_acc, train_fi_score = model_train(model = model, 
                                                            optimizer = optimizer, 
                                                            criterion = criterion, 
                                                            train_loder = train_loder)
        # 평가
        val_loss, val_acc, val_fi_score, = model_eval(model = model, 
                                                      criterion = criterion, 
                                                      val_loder = val_loder)
        
        # 학습률
        lr = get_lr(optimizer = optimizer)
        
        epoch_end_time = time.time()
        
        print(f'{now_fold_num + 1}fold, epoch: {epoch}, lr: {lr}, train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}, train_f1: {train_f1:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}, val_fi: {val_fi:.4f}, 학습시간: {epoch_end_time - epoch_start_time} \n')

        # 스케줄러
        scheduler.step(val_loss)
        
        # 모델 저장
        if besf_fi < val_fi_score:
            besf_fi = val_fi_score
            torch.save(model.state_dict(), model_dir + f'{now_fold_num + 1}fold_SWA_Efficientnet.pt')
    
    optimizer.swap_swa_sgd()
    torch.save(model.state_dict(), model_dir + f'{now_fold_num + 1}fold_total_SWA_Efficientnet.pt')
    
    fold_end_time = time.time()
    
    print(f'{now_fold_num + 1}fold 훈련 시간: {fold_end_time - fold_start_time} \n')
    gc.collect()
    torch.cuda.empty_cache()
    
total_end_time = time.time()
print(f'총 훈련 시간: {total_end_time - total_start_time}')

0it [00:00, ?it/s]

In [ ]:
'''

평가 및 예측

'''

skf = StratifiedKFold(n_splits = fold_num, random_state = seed, shuffle = True)

# 앙상블 데이터 저장용
train_oof = np.zeros((train_df.shape[0], 1))
submission_label_oof = np.zeros((submission.shape[0], 18))
submission_oof = np.zeros((submission.shape[0], 1))

# swa 앙상블 데이터 저장용
swa_train_oof = np.zeros((train_df.shape[0], 1))
swa_submission_label_oof = np.zeros((submission.shape[0], 18))
swa_submission_oof = np.zeros((submission.shape[0], 1))

# submission 데이터 정의
submission_dataset = CustomDataset(df = submission, 
                            transform = transform, 
                            train = False)

submission_loder = DataLoader(submission_dataset,
                        batch_size = batch_size,
                        shuffle = False,)

# 예측
for now_fold_num, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
    val_df = train_df.iloc[val_idx, :]
    
    val_dataset = CustomDataset(df = val_df, 
                                transform = transform, 
                                train = True)
    
    val_loder = DataLoader(val_dataset,
                            batch_size = batch_size,
                            shuffle = False,)
    
    
    # 모델 로드
    model = models.efficientnet_b3(pretrained=True)
    in_features = model.classifier[1].in_features
    model.classifier[1] = torch.nn.Linear(in_features, 18)
    model = model.to(device)
    model.load_state_dict(torch.load(model_dir + f'{now_fold_num + 1}fold_SWA_Efficientnet.pt', map_location = device))
    
    # 모델 로드
    swa_model = models.efficientnet_b3(pretrained=True)
    in_features = swa_model.classifier[1].in_features
    swa_model.classifier[1] = torch.nn.Linear(in_features, 18)
    swa_model = swa_model.to(device)
    swa_model.load_state_dict(torch.load(model_dir + f'{now_fold_num + 1}fold_total_SWA_Efficientnet.pt', map_location = device))
    
    # train_oof
    label_pred_li, swa_label_pred_li = [], []
    with torch.no_grad():
        for images, _, _, _, _ in val_loder:
            images = images.to(device)
            
            label = model(images).argmax(dim=-1)
            swa_label = swa_model(images).argmax(dim=-1)
            
            label_pred_li.extend(label.cpu().numpy())
            swa_label_pred_li.extend(swa_label.cpu().numpy())
    
    train_oof[val_idx, 0] = label_pred_li
    swa_train_oof[val_idx, 0] = swa_label_pred_li
    
    # submission_oof
    label_pred_li, swa_label_pred_li = [], []
    with torch.no_grad():
        for images in submission_loder:
            images = images.to(device)
            
            label = model(images).softmax(1)
            swa_label = swa_model(images).softmax(1)
            
            label_pred_li.append(label.cpu().numpy())
            swa_label_pred_li.append(swa_label.cpu().numpy())

    submission_label_oof += np.concatenate(label_pred_li) / fold_num
    swa_submission_label_oof += np.concatenate(swa_label_pred_li) / fold_num
    
    gc.collect()
    torch.cuda.empty_cache()
    
submission_oof[:, 0] = submission_label_oof.argmax(1).tolist()
swa_submission_oof[:, 0] = swa_submission_label_oof.argmax(1).tolist()

In [ ]:
col_li = ['label']

ensemble_train_df = pd.DataFrame(train_oof, columns = col_li).astype(int)
ensemble_swa_train_df = pd.DataFrame(swa_train_oof, columns = col_li).astype(int)

ensemble_submission_df = pd.DataFrame(submission_oof, columns = col_li).astype(int)
ensemble_swa_submission_df = pd.DataFrame(swa_submission_oof, columns = col_li).astype(int)

In [ ]:
'''

confusion_matrix and f1

'''

col_li = ['label']

for col in col_li:
    if col != 'ensemble_label':
        train_y_true = train_set[col].values
        train_y_pred = ensemble_train_df[col].values
        train_y_pred_swa = ensemble_swa_train_df[col].values
    
    else:
        train_y_true = train_set['label'].values
        train_y_pred = ensemble_train_df[col].values   
        train_y_pred_swa = ensemble_swa_train_df[col].values
    
    train_f1 = get_f1_score(y_true = train_y_true,  y_pred = train_y_pred)
    train_acc = get_acc_score(y_true = train_y_true, y_pred = train_y_pred)
    
    train_confusion_matrix = pd.DataFrame((confusion_matrix(y_true = train_y_true, y_pred = train_y_pred)))
    
    print(f'{col} train confusion_matrix')
    display(train_confusion_matrix.style.background_gradient(cmap='YlOrRd', axis = 0))
    
    print(f'train fi : {train_f1}, train acc: {train_acc} \n')
    
    swa_train_f1 = get_f1_score(y_true = train_y_true,  y_pred = train_y_pred_swa)
    swa_train_acc = get_acc_score(y_true = train_y_true, y_pred = train_y_pred_swa)
    
    swa_train_confusion_matrix = pd.DataFrame((confusion_matrix(y_true = train_y_true, y_pred = train_y_pred_swa)))
    
    print(f'{col} swa train confusion_matrix')
    display(swa_train_confusion_matrix.style.background_gradient(cmap='YlOrRd', axis = 0))
    
    print(f'swa train fi : {swa_train_f1}, swa train acc: {swa_train_acc} \n')

In [ ]:
# 예측에 사용할 라벨 결정
submission['ans'] = ensemble_submission_df['label'].astype(int).values

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(submission_data_dir, 'oof_adam_swa_Efficientnet_submission.csv'), index=False)
print('test inference is done!')

In [ ]:
submission.head()